In [1]:
import pandas as pd

In [2]:
import os
from pathlib import Path

# Define the directory path
data_dir = Path('data/store_database')

# Initialize an empty dictionary to store DataFrames
dfs = {}

# Iterate through Excel files in the directory
for file in data_dir.glob('*.xlsx'):
    # Read the Excel file into a DataFrame
    df = pd.read_excel(file)
    
    # Use the filename (without extension) as the key in the dictionary
    key = file.stem
    
    # Store the DataFrame in the dictionary
    dfs[key] = df


In [3]:
def process_dataframes(dfs):
    df_list = []
    for key, df in dfs.items():
        # Select the first 4 columns
        df = df.iloc[:, :4]
        
        # Rename the columns
        df.columns = ['item_id', 'brand', 'custom_label', 'current_quantity']
        
        df['store'] = key.split(" ")[0]
        # Update the DataFrame in the dictionary
        df_list.append(df)

    df_concat = pd.concat(df_list)
    return df_concat


df_store = process_dataframes(dfs)
df_store.to_csv('data/tables/store.csv', index=False)

In [5]:
df_store

,item_id,brand,custom_label,current_quantity,store
0,265315909866,MFD,A1,20,MFD
1,255523653607,KGF,A1,20,MFD
2,255523653608,KGF,A1,20,MFD
3,255523653611,KGF,A1,20,MFD
4,255523653614,KGF,A1,20,MFD
...,...,...,...,...,...
640089,156366861984,Unova,UKF-NAP-PWB1863,0,AST
640090,156366861993,Unova,UKF-NAP-PWB1864,0,AST
640091,204953308438,Unova,UKF-NAP-PWB1865,6,AST
640092,204953308437,Unova,UKF-NAP-PWB1866,0,AST
